In [14]:
from moviepy.video.io.VideoFileClip import VideoFileClip
import subprocess

filenametoanalyse = 'zoom_0'
mp3_audoifile = filenametoanalyse+'.mp3'
ogg_audoifile = filenametoanalyse+'.ogg'

videoclip = VideoFileClip(filenametoanalyse+'.mp4')#.subclip(0,240)
audioclip = videoclip.audio
audioclip.write_audiofile(mp3_audoifile, verbose=False) #, 16000, 2, 2000,"pcm_s16le",'32K')

command = 'ffmpeg -y -i ' 
command = command + mp3_audoifile
command = command + ' -c:a libopus ' 
command = command + ogg_audoifile
subprocess.check_call(command, shell=True)
print('ogg is saved')

chunk:   0%|                                                             | 70/41870 [00:00<01:00, 695.27it/s, now=None]

MoviePy - Writing audio in zoom_0.mp3


MoviePy - Done.
ogg is saved


In [15]:
# YANDEX работа с бакетами
import boto3

aws_access_key_id = 'ваш Идентификатор ключа доступа'
aws_secret_access_key = 'ваш секретный ключ от идентификатора'
endpoint = 'https://storage.yandexcloud.net'

session = boto3.session.Session()
s3 = session.client(
    service_name='s3',
    endpoint_url=endpoint, 
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
    region_name='ru-central1'  
)

# Создать новый бакет
#s3.create_bucket(Bucket='test123speech')

# Загрузить объекты в бакет

## Из строки
#s3.put_object(Bucket='bucket-name', Key='object_name', Body='TEST', StorageClass='COLD')

## Из файла
s3.upload_file(ogg_audoifile, 'speech331', ogg_audoifile)
#s3.upload_file('this_script.py', 'bucket-name', 'script/py_script.py')

# Получить список объектов в бакете
for key in s3.list_objects(Bucket='speech331')['Contents']:
    print(key['Key'])


sil_zoom_0.ogg
zoom_0.ogg


In [16]:
# Распознавание ДЛИННЫХ аудио YANDEX. Подходит для распознавания одноканального аудио небольшого размера.

import requests
import time
import json

# Укажите ваш API-ключ и ссылку на аудиофайл в Object Storage.
key = 'API-ключ'
filelink = 'https://storage.yandexcloud.net/speech331/' + ogg_audoifile

POST = "https://transcribe.api.cloud.yandex.net/speech/stt/v2/longRunningRecognize"

body ={
    "config": {
        "specification": {
            "languageCode": "ru-RU",
            "model": "deferred-general"
        }
    },
    "audio": {
        "uri": filelink
    }
}

# Если вы хотите использовать IAM-токен для аутентификации, замените Api-Key на Bearer.
header = {'Authorization': 'Api-Key {}'.format(key)}

# Отправить запрос на распознавание.
req = requests.post(POST, headers=header, json=body)
data = req.json()
print(data)

id = data['id']
step=30
tt=0
# Запрашивать на сервере статус операции, пока распознавание не будет завершено.
while True:

    time.sleep(step)
    tt=tt+step

    GET = "https://operation.api.cloud.yandex.net/operations/{id}"
    req = requests.get(GET.format(id=id), headers=header)
    req = req.json()

    if req['done']: break
    print("Not ready "+ str(tt))
    
# Показать полный ответ сервера в формате JSON.
#print("Response:")
#print(json.dumps(req, ensure_ascii=False, indent=2))

# Показать только текст из результатов распознавания.
print("Text chunks:")
for chunk in req['response']['chunks']:
    print(chunk['alternatives'][0]['text'])

{'done': False, 'id': 'e036rp5lhfv9ptgqd3gu', 'createdAt': '2021-01-02T18:09:52Z', 'createdBy': 'ajemboq8tmf6fksl9svj', 'modifiedAt': '2021-01-02T18:09:52Z'}
Not ready 30
Not ready 60
Not ready 90
Not ready 120
Not ready 150
Not ready 180
Not ready 210
Not ready 240
Not ready 270
Not ready 300
Not ready 330
Not ready 360
Not ready 390
Not ready 420
Text chunks:
Так еще раз
Так еще раз
А
А
Прям себя что на свистните контейнер закрытый
Прям себя что на свистните контейнер закрытый
В котором находится молекула часть маленького находится
В котором находится молекула часть маленького находится
В жидкой фазе
В жидкой фазе
Часть находится
Часть находится
Газообразная фаза
Газообразная фаза
Так как то анекдот
Так как то анекдот
И тут поменьше
И тут поменьше
Да спокойно
Да спокойно
Теперь
Теперь
Обозначила
Обозначила
Количество молекул за
Количество молекул за
Большой
Большой
Вообще
Вообще
Коничева
Коничева
Маньяков
Маньяков
Количество молитва
А как количество молитва
Монах
Монах
А моли это пос

In [25]:
# Показать полный ответ сервера в формате JSON.
print("Response:")
print(json.dumps(req, ensure_ascii=False, indent=2))

Response:
{
  "done": true,
  "response": {
    "@type": "type.googleapis.com/yandex.cloud.ai.stt.v2.LongRunningRecognitionResponse",
    "chunks": [
      {
        "alternatives": [
          {
            "words": [
              {
                "startTime": "1.840s",
                "endTime": "2.180s",
                "word": "так",
                "confidence": 1
              },
              {
                "startTime": "2.260s",
                "endTime": "2.440s",
                "word": "еще",
                "confidence": 1
              },
              {
                "startTime": "2.480s",
                "endTime": "3.200s",
                "word": "раз",
                "confidence": 1
              }
            ],
            "text": "Так еще раз",
            "confidence": 1
          }
        ],
        "channelTag": "1"
      },
      {
        "alternatives": [
          {
            "words": [
              {
                "startTime": "1.840s",
      

In [20]:
# Показать только определенные поля из ответа.
#print("Text chunks:")
for chunk in req['response']['chunks']:
    if chunk['channelTag'][0] is not '2':
        print(chunk['alternatives'][0]['words'][0]['startTime']+" "+
              chunk['alternatives'][0]['words'][0]['endTime'])
        #print(chunk['alternatives'][0]['text'])
    #print(chunk['channelTag'][0])

1.840s 2.180s
3.480s 4.220s
5.130s 5.390s
8.890s 8.930s
12.850s 12.890s
15.210s 15.430s
18.430s 19.050s
23.250s 23.430s
25.040s 25.120s
27.120s 27.320s
30.130s 31.270s
31.240s 32.560s
33.640s 34.060s
36.160s 37.240s
40.240s 41.500s
42.400s 43.780s
45.060s 46.440s
53.420s 53.860s
56.300s 57.080s
57.530s 57.590s
60.690s 61.670s
61.660s 62.580s
64.550s 65.650s
66.630s 67.470s
69.460s 70.620s
71s 71.500s
74.630s 75.570s
78.490s 79.350s
79.550s 80.570s
81.220s 82.260s
84.630s 85.830s
86.730s 86.830s
89.100s 90.200s
90.020s 91.220s
92.580s 93.640s
95.220s 95.260s
100.910s 101.210s
102.180s 102.900s
104.590s 105.630s
105.720s 106.080s
108.650s 109.830s
110.630s 111.770s
113.010s 114.050s
115.110s 115.490s
116.950s 117.550s
118.600s 119.400s
120.670s 121.750s
122.930s 123.230s
124.660s 125.480s
125.380s 126.240s
128.330s 129.690s
140.170s 140.350s
143.150s 143.330s
146.660s 147.580s
148.590s 149.690s
150.890s 152.030s
153.470s 154.190s
156.770s 157.570s
160.250s 160.390s
163.500s 164.600s
165.

In [27]:
file1 = open("afterYandex.txt","w") 
for chunk in req['response']['chunks']:
    if chunk['channelTag'][0] is not '2':
        file1.write((chunk['alternatives'][0]['words'][0]['startTime'] +" "+
              chunk['alternatives'][0]['words'][0]['endTime'] +"\n").replace('s', ''))
file1.close() 